Провести сравнение RNN, LSTM, GRU на датасете отзывов (из предыдущих занятий/материалов)

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import emoji
from nltk import word_tokenize
from string import punctuation, ascii_letters
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
from collections import Counter
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import regularizers
from gensim.models import Word2Vec

In [22]:
MAX_WORDS = 5000
MAX_LEN = 20
EMB_SIZE = 100

In [23]:
morph = MorphAnalyzer(lang='ru')

In [24]:
stop_words = set(get_stop_words('ru'))

In [25]:
punctuation = set(punctuation).union((' ', '«', '»', '—', '–', '“', '”', '…'))

In [26]:
cyrillic_letters = set([chr(i) for i in range(ord('а'), ord('я') + 1)] +
                       [chr(i) for i in range(ord('А'), ord('Я') + 1)] +
                       ['ё', 'Ё'])

In [27]:
data = pd.read_excel('../data/summer_reviews.xls')
data.columns = ['rating', 'content', 'date']

In [28]:
data['target'] = (data.rating > 3).astype(int)

In [29]:
def preprocess(text):
    tokens = word_tokenize(text)
    result = []
    for token in tokens:
        if (set(token).intersection(cyrillic_letters)
            or set(token).intersection(set(ascii_letters))
            or token in emoji.UNICODE_EMOJI):
            result.append(token)
    tokens = [token.lower() for token in result if token.lower() not in stop_words]
    tokens = [token for token in tokens if token in emoji.UNICODE_EMOJI or len(token) >  1]
    tokens = [morph.parse(token)[0].normal_form for token in tokens]
    return tokens
        

In [30]:
data['processed'] = data.content.apply(lambda x: " ".join(preprocess(str(x))))

In [31]:
X_train, X_val, y_train, y_val = train_test_split(data['processed'], data['target'], test_size=0.1,
                                                    random_state=0, stratify=data['target'])

In [32]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=MAX_WORDS, oov_token='UNK')

In [33]:
tokenizer.fit_on_texts(X_train)

In [34]:
X_train = pad_sequences(tokenizer.texts_to_sequences(X_train),
                        maxlen=MAX_LEN,
                        padding='post',
                        truncating='post')

In [35]:
X_val = pad_sequences(tokenizer.texts_to_sequences(X_val),
                        maxlen=MAX_LEN,
                        padding='post',
                        truncating='post')

In [36]:
vocab = tokenizer.index_word

### RNN

In [37]:
inputs = tf.keras.layers.Input(shape=(MAX_LEN,))
embeddings = tf.keras.layers.Embedding(input_dim=len(vocab),
                                       output_dim=EMB_SIZE,
                                       activity_regularizer=regularizers.l2(1e-6))(inputs)
rnn = tf.keras.layers.SimpleRNN(128, recurrent_dropout=0.2, activation='relu')(embeddings)
dense = tf.keras.layers.Dense(64, activation='relu')(rnn)
outputs = tf.keras.layers.Dense(1, activation='sigmoid')(dense)

In [38]:
model = tf.keras.Model(inputs=inputs, outputs=outputs)

In [39]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

In [40]:
model.compile(optimizer=optimizer,
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [41]:
model.fit(X_train, y_train, 
          validation_data=(X_val, y_val),
          batch_size=256,
          epochs=10)

Train on 18593 samples, validate on 2066 samples
Epoch 1/10
18593/18593 [==============================] - 2s 134us/sample - loss: 0.4494 - accuracy: 0.8169 - val_loss: 0.2912 - val_accuracy: 0.8901
Epoch 2/10
18593/18593 [==============================] - 2s 97us/sample - loss: 0.2870 - accuracy: 0.8735 - val_loss: 0.2519 - val_accuracy: 0.9085
Epoch 3/10
18593/18593 [==============================] - 2s 97us/sample - loss: 0.2626 - accuracy: 0.8700 - val_loss: 0.2494 - val_accuracy: 0.8964
Epoch 4/10
18593/18593 [==============================] - 2s 96us/sample - loss: 0.2571 - accuracy: 0.8639 - val_loss: 0.2471 - val_accuracy: 0.8393
Epoch 5/10
18593/18593 [==============================] - 2s 97us/sample - loss: 0.2714 - accuracy: 0.8654 - val_loss: 0.2765 - val_accuracy: 0.8533
Epoch 6/10
18593/18593 [==============================] - 2s 95us/sample - loss: 0.2436 - accuracy: 0.9023 - val_loss: 0.2636 - val_accuracy: 0.9080
Epoch 7/10
18593/18593 [==============================] 

### Bidirectional LSTM

In [42]:
inputs = tf.keras.layers.Input(shape=(MAX_LEN,))
embeddings = tf.keras.layers.Embedding(input_dim=len(vocab),
                                       output_dim=EMB_SIZE,
                                       activity_regularizer=regularizers.l2(1e-6))(inputs)
lstm = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64))(embeddings)
dense = tf.keras.layers.Dense(64, activation='relu')(lstm)
outputs = tf.keras.layers.Dense(1, activation='sigmoid')(dense)

In [43]:
model = tf.keras.Model(inputs=inputs, outputs=outputs)

In [44]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

In [45]:
model.compile(optimizer=optimizer,
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [46]:
model.fit(X_train, y_train, 
          validation_data=(X_val, y_val),
          batch_size=256,
          epochs=10)

Train on 18593 samples, validate on 2066 samples
Epoch 1/10
18593/18593 [==============================] - 5s 280us/sample - loss: 0.3586 - accuracy: 0.8510 - val_loss: 0.2071 - val_accuracy: 0.9071
Epoch 2/10
18593/18593 [==============================] - 3s 179us/sample - loss: 0.1969 - accuracy: 0.9169 - val_loss: 0.1813 - val_accuracy: 0.9148
Epoch 3/10
18593/18593 [==============================] - 3s 175us/sample - loss: 0.1702 - accuracy: 0.9325 - val_loss: 0.1871 - val_accuracy: 0.9153
Epoch 4/10
18593/18593 [==============================] - 3s 180us/sample - loss: 0.1510 - accuracy: 0.9441 - val_loss: 0.1946 - val_accuracy: 0.9158
Epoch 5/10
18593/18593 [==============================] - 3s 176us/sample - loss: 0.1376 - accuracy: 0.9487 - val_loss: 0.2132 - val_accuracy: 0.9124
Epoch 6/10
18593/18593 [==============================] - 3s 176us/sample - loss: 0.1255 - accuracy: 0.9525 - val_loss: 0.2255 - val_accuracy: 0.9095
Epoch 7/10
18593/18593 [===========================

### GRU

In [47]:
inputs = tf.keras.layers.Input(shape=(MAX_LEN,))
embeddings = tf.keras.layers.Embedding(input_dim=len(vocab),
                                       output_dim=EMB_SIZE,
                                       activity_regularizer=regularizers.l2(1e-6))(inputs)
gru = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(64))(embeddings)
dense = tf.keras.layers.Dense(64, activation='relu')(gru)
outputs = tf.keras.layers.Dense(1, activation='sigmoid')(dense)

In [48]:
model = tf.keras.Model(inputs=inputs, outputs=outputs)

In [49]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

In [50]:
model.compile(optimizer=optimizer,
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [51]:
model.fit(X_train, y_train, 
          validation_data=(X_val, y_val),
          batch_size=256,
          epochs=10)

Train on 18593 samples, validate on 2066 samples
Epoch 1/10
18593/18593 [==============================] - 5s 262us/sample - loss: 0.3545 - accuracy: 0.8509 - val_loss: 0.2076 - val_accuracy: 0.9037
Epoch 2/10
18593/18593 [==============================] - 3s 157us/sample - loss: 0.1943 - accuracy: 0.9201 - val_loss: 0.1841 - val_accuracy: 0.9158
Epoch 3/10
18593/18593 [==============================] - 3s 154us/sample - loss: 0.1660 - accuracy: 0.9343 - val_loss: 0.1911 - val_accuracy: 0.9153
Epoch 4/10
18593/18593 [==============================] - 3s 162us/sample - loss: 0.1496 - accuracy: 0.9437 - val_loss: 0.1923 - val_accuracy: 0.9163
Epoch 5/10
18593/18593 [==============================] - 3s 152us/sample - loss: 0.1353 - accuracy: 0.9498 - val_loss: 0.2164 - val_accuracy: 0.9076
Epoch 6/10
18593/18593 [==============================] - 3s 155us/sample - loss: 0.1241 - accuracy: 0.9542 - val_loss: 0.2186 - val_accuracy: 0.9105
Epoch 7/10
18593/18593 [===========================

Как видим, на валидации у всех моделей результат схожий. Если смотреть на лучшую эпоху, то небольшое преимущество у GRU с результатом `0.9163`.